In [1]:
import pandas as pd
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.datasets import make_multilabel_classification
from skmultilearn.model_selection import IterativeStratification
import random
import csv
import glob

# Loads and manipulates

In [2]:
orb = pd.read_csv("/neurospin/dico/data/bv_databases/human/partially_labeled/orbital_paterns/Troiani/HCP_OFC_for_JFM - Types_and_Subtypes.csv", index_col=0)
orb = orb.dropna()
orb["Right"] = orb["Right"].astype(int)
orb["Left"] = orb["Left"].astype(int)

In [3]:
participants_file = "/neurospin/dico/data/bv_databases/human/not_labeled/hcp/participants.csv"
participants_unrestricted = pd.read_csv(participants_file)
participants_unrestricted = participants_unrestricted[['Subject', "Gender"]]
print(len(participants_unrestricted))
participants_unrestricted.head()

1206


,Subject,Gender
0,100004,M
1,100206,M
2,100307,F
3,100408,M
4,100610,M


In [4]:
participants_file = "/neurospin/dico/jchavas/RESTRICTED_jchavas_1_18_2022_3_17_51.csv"
participants = pd.read_csv(participants_file)
participants = participants[['Subject', 'ZygosityGT', 'Family_ID']]
print(len(participants))
participants.head()

1206


,Subject,ZygosityGT,Family_ID
0,100004,,52259_82122
1,100206,,56037_85858
2,100307,MZ,51488_81352
3,100408,MZ,51730_81594
4,100610,DZ,52813_82634


In [5]:
participants = pd.merge(participants_unrestricted, participants)
participants.loc[(participants['ZygosityGT']== " "), 'ZygosityGT'] = 'NotTwin'  
participants['Subject'] = participants['Subject'].astype('string')
print(len(participants))
participants.head()

1206


,Subject,Gender,ZygosityGT,Family_ID
0,100004,M,NotTwin,52259_82122
1,100206,M,NotTwin,56037_85858
2,100307,F,MZ,51488_81352
3,100408,M,MZ,51730_81594
4,100610,M,DZ,52813_82634


In [6]:
participants.dtypes

Subject       string[python]
Gender                object
ZygosityGT            object
Family_ID             object
dtype: object

In [7]:
treated_subjects = glob.glob("/neurospin/dico/data/bv_databases/human/not_labeled/hcp/hcp/*[!.minf]")
treated_subjects = [x.split('/')[-1] for x in treated_subjects]
treated_subjects = [x for x in treated_subjects if 'database' not in x]
print(treated_subjects[:5])
len(treated_subjects)
participants = participants[participants['Subject'].isin(treated_subjects)]
print(len(participants))
set(treated_subjects) - set(participants['Subject'])

['210112', '579665', '922854', '517239', '329440']
1113


{'142626'}

In [12]:
print(len(orb))
print(orb.dtypes)
print(orb[["Right"]].value_counts(dropna=False))
print(orb[["Left"]].value_counts(dropna=False))
orb.index = orb.index.astype(str)

577
Right             int64
Right Subtype    object
Left              int64
Left Subtype     object
dtype: object
Right
1        339
2        120
3         99
4         19
Name: count, dtype: int64
Left
1       281
2       159
3       101
4        36
Name: count, dtype: int64


In [13]:
orb.head()

,Right,Right Subtype,Left,Left Subtype
100307,1,B,2,F
100408,1,N,2,E
100610,1,A,1,A
101006,1,B,1,A
101410,2,A,2,I


In [14]:
orb.index

Index(['100307', '100408', '100610', '101006', '101410', '102311', '102816',
       '103010', '103515', '103818',
       ...
       '966975', '969476', '971160', '972566', '973770', '984472', '987983',
       '990366', '995174', '996782'],
      dtype='object', length=577)

In [16]:
set(orb.index) - set(treated_subjects)

set()

In [18]:
orb.index.isin(set(treated_subjects) - set(participants['Subject'])).sum()

0

In [20]:
orb.head()

,Right,Right Subtype,Left,Left Subtype
100307,1,B,2,F
100408,1,N,2,E
100610,1,A,1,A
101006,1,B,1,A
101410,2,A,2,I


In [21]:
participants.head()

,Subject,Gender,ZygosityGT,Family_ID
1,100206,M,NotTwin,56037_85858
2,100307,F,MZ,51488_81352
3,100408,M,MZ,51730_81594
4,100610,M,DZ,52813_82634
5,101006,F,NotTwin,51283_52850_81149


In [28]:
orb = pd.merge(participants, orb, left_on='Subject', right_index=True).set_index('Subject')

# Makes split without considering twins

In [36]:
orb[["Gender"]].value_counts()

Gender
F         349
M         228
Name: count, dtype: int64

In [37]:
orb.query("Right==1 and Left==1 and Gender=='M'")

,Gender,ZygosityGT,Family_ID,Right,Right Subtype,Left,Left Subtype
Subject,,,,,,,
100610,M,DZ,52813_82634,1,A,1,A
103010,M,MZ,55895_85715,1,A,1,A
106824,M,DZ,56183_86002,1,I,1,A
107422,M,MZ,54628_84450,1,B,1,A
108020,M,DZ,56200_86019,1,C,1,A
...,...,...,...,...,...,...,...
832651,M,DZ,55904_85724,1,N,1,E
871964,M,NotTwin,51852_81716,1,N,1,C
898176,M,DZ,51749_81613,1,F,1,N


In [38]:
def print_frequencies(df):
    for right in df['Right'].unique():
        for left in df['Left'].unique():
            for gender in df['Gender'].unique():
                freq = df.query("Right==@right and Left==@left and Gender==@gender")
                print(f"{right}, {left}, {gender}: {len(freq)}")

In [39]:
print_frequencies(orb)

1, 2, F: 50
1, 2, M: 36
1, 1, F: 111
1, 1, M: 67
1, 3, F: 49
1, 3, M: 14
1, 4, F: 7
1, 4, M: 5
2, 2, F: 16
2, 2, M: 28
2, 1, F: 27
2, 1, M: 27
2, 3, F: 7
2, 3, M: 2
2, 4, F: 5
2, 4, M: 8
3, 2, F: 13
3, 2, M: 11
3, 1, F: 27
3, 1, M: 14
3, 3, F: 17
3, 3, M: 7
3, 4, F: 8
3, 4, M: 2
4, 2, F: 3
4, 2, M: 2
4, 1, F: 5
4, 1, M: 3
4, 3, F: 3
4, 3, M: 2
4, 4, F: 1
4, 4, M: 0


In [44]:
def iterative_split(df, folds, n_splits, stratify_columns):
    """Custom iterative train test split which
    'maintains balanced representation with respect
    to order-th label combinations.'

    From https://madewithml.com/courses/mlops/splitting/#stratified-split
    """
    # One-hot encode the stratify columns and concatenate them
    one_hot_cols = [pd.get_dummies(df[col]) for col in stratify_columns]
    one_hot_cols = pd.concat(one_hot_cols, axis=1).to_numpy()
    stratifier = IterativeStratification(
        n_splits=n_splits, order=len(stratify_columns), sample_distribution_per_fold=folds)
    folds = []
    for indices in stratifier.split(df.to_numpy(), one_hot_cols):
        folds.append(df.iloc[indices[1]])
    return folds

In [45]:
folds = iterative_split(orb, [0.1,]*10, 10, ['Right', 'Left', 'Gender'])

In [46]:
print(len(folds))

10


In [47]:
def print_results(parent, folds, col):

    # For each conbination of labels, prints the number of rows for each fold
    # having this combination
    total_errors = 0
    n_splits = len(folds)
    print("query   : #rows      : #rows per fold\n")

    for col0 in parent[col[0]].unique():
        for col1 in parent[col[1]].unique():
            for col2 in parent[col[2]].unique():
                df = parent.query(f"{col[0]}==@col0 and {col[1]}==@col1 and {col[2]}==@col2")
                len_query = len(df)
                print(f"{col0}, {col1}, {col2} : total = {len_query} : per fold =", end = ' ')
                for fold in folds:
                    df0 = fold.query(f"{col[0]}==@col0 and {col[1]}==@col1 and {col[2]}==@col2")
                    len_query_fold = len(df0)
                    if abs(len_query_fold-len_query/n_splits) >= 1:
                        total_errors += 1
                    print(f"{len_query_fold} -", end= ' ')
                print("")

    # Prints the statistics and the number of stratification errors
    expected_total_length = len(parent)
    total_length = 0
    total_mismatches = 0
    print("\nlengths of folds : ", end = ' ')
    for fold in folds:
        len_fold = len(fold)
        print(len_fold, end=' ')
        total_length += len_fold
        if abs(len_fold-expected_total_length/n_splits) >= 1:
            total_mismatches += 1
    print(f"\nExpected total_length = {expected_total_length}")
    print(f"Effective total_length = {total_length}")

    print(f"total number of stratification errors: {total_errors}")
    print(f"total number of mismatched fold sizes : {total_mismatches}")

In [48]:
print_results(orb, folds, ['Right', 'Left', 'Gender'])

query   : #rows      : #rows per fold

1, 2, F : total = 50 : per fold = 5 - 5 - 5 - 5 - 5 - 5 - 5 - 5 - 5 - 5 - 
1, 2, M : total = 36 : per fold = 4 - 4 - 3 - 3 - 4 - 4 - 3 - 4 - 3 - 4 - 
1, 1, F : total = 111 : per fold = 10 - 11 - 12 - 13 - 10 - 10 - 12 - 11 - 11 - 11 - 
1, 1, M : total = 67 : per fold = 7 - 6 - 6 - 7 - 7 - 7 - 7 - 6 - 7 - 7 - 
1, 3, F : total = 49 : per fold = 5 - 5 - 5 - 5 - 5 - 5 - 5 - 5 - 5 - 4 - 
1, 3, M : total = 14 : per fold = 1 - 1 - 2 - 1 - 2 - 2 - 1 - 2 - 1 - 1 - 
1, 4, F : total = 7 : per fold = 1 - 1 - 0 - 0 - 1 - 1 - 1 - 0 - 1 - 1 - 
1, 4, M : total = 5 : per fold = 1 - 0 - 1 - 0 - 0 - 0 - 0 - 1 - 1 - 1 - 
2, 2, F : total = 16 : per fold = 2 - 2 - 2 - 1 - 1 - 2 - 2 - 1 - 1 - 2 - 
2, 2, M : total = 28 : per fold = 2 - 3 - 3 - 3 - 3 - 3 - 3 - 3 - 3 - 2 - 
2, 1, F : total = 27 : per fold = 3 - 3 - 3 - 3 - 2 - 3 - 3 - 3 - 2 - 2 - 
2, 1, M : total = 27 : per fold = 3 - 2 - 3 - 3 - 3 - 3 - 2 - 3 - 3 - 2 - 
2, 3, F : total = 7 : per fold = 1 - 1 - 1 - 1 - 0 -

In [49]:
def iterative_split_through_sorting_shuffle(df, n_splits, stratify_columns, random_state):
    """Custom iterative train test split which
    maintains balanced representation.
    """
    # Dataframe random row shuffle + sorting according to stratify_columns
    sorted = df.sample(frac=1, random_state=random_state).sort_values(stratify_columns)
    # for each fold, we take one row every n_splits rows
    folds = [sorted.iloc[i::n_splits, :] for i in range(n_splits)]
    # Further shuffling
    folds = [fold.sample(frac=1, random_state=random_state) for fold in folds]
    random.Random(random_state).shuffle(folds)
    return folds

In [52]:
folds = iterative_split_through_sorting_shuffle(orb, 10, ['Right', 'Left', 'Gender'], 1)

In [53]:
print_results(orb, folds, ['Right', 'Left', 'Gender'])

query   : #rows      : #rows per fold

1, 2, F : total = 50 : per fold = 5 - 5 - 5 - 5 - 5 - 5 - 5 - 5 - 5 - 5 - 
1, 2, M : total = 36 : per fold = 3 - 4 - 4 - 3 - 3 - 4 - 4 - 3 - 4 - 4 - 
1, 1, F : total = 111 : per fold = 11 - 11 - 11 - 11 - 11 - 11 - 12 - 11 - 11 - 11 - 
1, 1, M : total = 67 : per fold = 7 - 6 - 6 - 7 - 7 - 7 - 6 - 7 - 7 - 7 - 
1, 3, F : total = 49 : per fold = 5 - 5 - 5 - 5 - 5 - 4 - 5 - 5 - 5 - 5 - 
1, 3, M : total = 14 : per fold = 2 - 1 - 1 - 1 - 2 - 2 - 1 - 2 - 1 - 1 - 
1, 4, F : total = 7 : per fold = 0 - 1 - 1 - 1 - 0 - 1 - 1 - 0 - 1 - 1 - 
1, 4, M : total = 5 : per fold = 1 - 1 - 0 - 1 - 1 - 0 - 0 - 1 - 0 - 0 - 
2, 2, F : total = 16 : per fold = 2 - 2 - 1 - 2 - 2 - 2 - 1 - 2 - 1 - 1 - 
2, 2, M : total = 28 : per fold = 3 - 2 - 3 - 2 - 3 - 3 - 3 - 3 - 3 - 3 - 
2, 1, F : total = 27 : per fold = 2 - 2 - 3 - 2 - 3 - 3 - 3 - 3 - 3 - 3 - 
2, 1, M : total = 27 : per fold = 3 - 3 - 3 - 3 - 2 - 2 - 3 - 2 - 3 - 3 - 
2, 3, F : total = 7 : per fold = 0 - 1 - 1 - 1 - 0 -

# Makes splits by considering zygosity and gender

In [ ]:
acc.head()

In [ ]:
acc.groupby(["DATABASE","SEX", "Right_PCS"]).size()

In [ ]:
def sampling_subdataset_based_on_full_dataset(full, sub, n_splits):
    """Sampling of one fold the subdataset sub such that
    for each combination, the proportion equals the proportion of the full dataset divided by n_splits
    """
    
    total = 0
    test = pd.DataFrame()
    full_counts = full.value_counts(["DATABASE","SEX", "Right_PCS"]).reset_index(name='counts')
    for row in full_counts.iterrows():
        row = row[1]
        total_comb = row['counts']
        df = sub.query(f"DATABASE=='{row['DATABASE']}' and SEX=='{row['SEX']}' and Right_PCS=='{row['Right_PCS']}'")

        if len(test) < total/n_splits:
            df = df.iloc[0:(int(total_comb/n_splits)+1),:]
        else:
            df = df.iloc[0:int(total_comb/n_splits),:]
            
        test = pd.concat([test, df])
        total_sub = len(df)
        total += total_comb
        print(f"{row['DATABASE']}, {row['SEX']}, {row['Right_PCS']}: {total_comb}, {total_sub}")

    return test

In [ ]:
test = sampling_subdataset_based_on_full_dataset(acc, acc0, 5)

In [ ]:
print(len(test))
test.head()

In [ ]:
# acc0_reduced = acc0.merge(test0, how='outer', indicator=True).query('_merge != "both"').drop('_merge', axis=1)

In [ ]:
folds_test = iterative_split_through_sorting_shuffle(test, 2, ['DATABASE', 'SEX', 'Right_PCS'], 3)

In [ ]:
print_results(test, folds_test,['DATABASE', 'SEX', 'Right_PCS'])

In [ ]:
acc_train = acc.merge(test, how='outer', indicator=True).query('_merge != "both"').drop('_merge', axis=1)

In [ ]:
folds_train = iterative_split_through_sorting_shuffle(acc_train, 8, ['DATABASE', 'SEX', 'Right_PCS'], 3)

In [ ]:
print_results(acc_train, folds_train,['DATABASE', 'SEX', 'Right_PCS'])

In [ ]:
folds = folds_train + folds_test
len(folds)

In [ ]:
print_results(acc, folds,['DATABASE', 'SEX', 'Right_PCS'])

In [ ]:
(~folds_test[1]['long_name'].isin(acc0['long_name'])).sum()

In [ ]:
folds_test[0].dtypes

# Saving results

In [ ]:
save_path = "/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns"
prefix_test = "ACCpatterns_subjects_test"
prefix_train = "ACCpatterns_subjects_train"

In [ ]:
for i in range(len(folds_test)):
    folds_test[i]["long_name"].to_csv(
        f"{save_path}/{prefix_test}_split_{i}.csv",
        index=False,
        header=False,
        quoting=csv.QUOTE_ALL)

In [ ]:
for i in range(len(folds_train)):
    folds_train[i]["long_name"].to_csv(
        f"{save_path}/{prefix_train}_split_{i}.csv",
        index=False,
        header=False,
        quoting=csv.QUOTE_ALL)